In [1]:
!pip install langchain==0.0.197
#!pip install langchain==0.0.165

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.3 MB/s eta 0:00:00


In [44]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=bc426b7e58cadf7859cb491d8eaf26f7ca525bdca4ead54cd22bd0e03505423a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
!pip install gpt4all==0.3.6
#!pip install gpt4all==0.3.4
#!pip uninstall pygpt4all==1.1.0 -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.9 MB/s eta 0:00:00


In [3]:
import langchain
print(langchain.__version__)

0.0.197


In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [45]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

In [46]:
embeddings = HuggingFaceEmbeddings(model_name = "paraphrase-MiniLM-L6-v2")

In [47]:
text = "Artificial intelligence learns from data"

In [48]:
query_result= embeddings.embed_query(text)

In [49]:
doc_result = embeddings.embed_documents([text, "this is not about AI"])

In [50]:
doc_result[0][0:10]

[-0.011978083290159702,
 -0.5602169632911682,
 -0.05175710842013359,
 -0.6471124887466431,
 0.5174511671066284,
 0.0029370635747909546,
 0.5730504989624023,
 0.08426259458065033,
 -0.09116951376199722,
 0.3031725585460663]

In [51]:
len(doc_result)

2

# Create Vector Store in-memory

In [52]:
import pandas as pd

In [53]:
path = "/content/drive/MyDrive/Colab Notebooks/ds_projects/articles.csv"

In [54]:
articles = pd.read_csv(path)

In [55]:
articles.head(2)

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."
1,Conor Dewey,1.4K,7,https://towardsdatascience.com/python-for-data...,Python for Data Science: 8 Concepts You May Ha...,If you’ve ever found yourself looking up the s...


In [56]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import SKLearnVectorStore

In [57]:
from langchain.document_loaders import CSVLoader

In [58]:
documents = CSVLoader(path).load()

In [59]:
#documents

In [60]:
#documents = articles["text"]

# Create Chunks of data

In [84]:
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

In [85]:
docs = text_splitter.split_documents(documents)

# Create Vectorstore - InMemory

In [86]:
import tempfile
import os

In [87]:
persistant_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/ds_projects/", "in_memory_database.parquet")

In [88]:
print(persistant_path)

/content/drive/MyDrive/Colab Notebooks/ds_projects/in_memory_database.parquet


In [89]:
vector_store = SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_path=persistant_path,
    serializer="parquet"
)

In [90]:
query = "Give me some examples of chatbots"
docs = vector_store.similarity_search(query)

In [91]:
print(docs[0].page_content)

author: Matt Schlicht
claps: 5K
reading_time: 11
link: https://chatbotsmagazine.com/the-complete-beginner-s-guide-to-chatbots-8280b7b906ca?source=tag_archive---------3----------------
title: The Complete Beginner’s Guide To Chatbots – Chatbots Magazine
text: What are chatbots? Why are they such a big opportunity? How do they work? How can I build one? How can I meet other people interested in chatbots?
These are the questions we’re going to answer for you right now.
Ready? Let’s do this.
(Do you work in ecommerce? Stop reading and click here, we made something for you.)
(p.s. here is where I believe the future of bots is headed, you will probably disagree with me at first.)
(p.p.s. My newest guide about conversational commerce is up, I think you’ll find it super interesting.)
A chatbot is a service, powered by rules and sometimes artificial intelligence, that you interact with via a chat interface. The service could be any number of things, ranging from functional to fun, and it could 

In [69]:
query = "Where does deep learning fail ? "
docs = vector_store.similarity_search(query)

In [70]:
print(docs[0].page_content)

author: Bargava
claps: 11.8K
reading_time: 3
link: https://towardsdatascience.com/how-to-learn-deep-learning-in-6-months-e45e40ef7d48?source=tag_archive---------5----------------
title: How to learn Deep Learning in 6 months – Towards Data Science
text: It is quite possible to learn, follow and contribute to state-of-art work in deep learning in about 6 months’ time. This article details out the steps to achieve that.
Pre-requisites
- You are willing to spend 10–20 hours per week for the next 6 months- You have some programming skills. You should be comfortable to pick up Python along the way. And cloud. (No background in Python and cloud assumed).- Some math education in the past (algebra, geometry etc). - Access to internet and computer.
Step 1
We learn driving a car — by driving. Not by learning how the clutch and the internal combustion engine work. Atleast not initially. When learning deep learning, we will follow the same top-down approach.
Do the fast.ai course — Practical Deep 

# Open Source LLM Download -> GPT4ALL

In [ ]:
llm_download_model_path = "/content/drive/MyDrive/Colab Notebooks/ds_projects/llm_model/ggml-gpt4all-l13b-snoozy.bin"

In [ ]:
llm_download_model_path

'/content/drive/MyDrive/Colab Notebooks/ds_projects/llm_model/ggml-gpt4all-l13b-snoozy.bin'

In [ ]:
# import requests

# from pathlib import Path
# from tqdm import tqdm

# Path(llm_download_model_path).parent.mkdir(parents=True, exist_ok=True)

# # Example model. Check https://github.com/nomic-ai/gpt4all for the latest models.
# url = 'http://gpt4all.io/models/ggml-gpt4all-l13b-snoozy.bin'

# # send a GET request to the URL to download the file. Stream since it's large
# response = requests.get(url, stream=True)

# # open the file in binary mode and write the contents of the response to it in chunks
# # This is a large file, so be prepared to wait.
# with open(llm_download_model_path, 'wb') as f:
#     for chunk in tqdm(response.iter_content(chunk_size=8192)):
#         if chunk:
#             f.write(chunk)

# print("Download Done!!")

GPT4ALL Try

In [ ]:
llm_download_model_dir = "/content/drive/MyDrive/Colab Notebooks/ds_projects/llm_model/"

In [ ]:
llm_download_model_path_small = "/content/drive/MyDrive/Colab Notebooks/ds_projects/llm_model/orca-mini-3b.ggmlv3.q4_0.bin"

In [ ]:
# import requests

# from pathlib import Path
# from tqdm import tqdm

# Path(llm_download_model_path_small).parent.mkdir(parents=True, exist_ok=True)

# # Example model. Check https://github.com/nomic-ai/gpt4all for the latest models.
# url = 'http://gpt4all.io/models/orca-mini-3b.ggmlv3.q4_0.bin'

# # send a GET request to the URL to download the file. Stream since it's large
# response = requests.get(url, stream=True)

# # open the file in binary mode and write the contents of the response to it in chunks
# # This is a large file, so be prepared to wait.
# with open(llm_download_model_path_small, 'wb') as f:
#     for chunk in tqdm(response.iter_content(chunk_size=8192)):
#         if chunk:
#             f.write(chunk)

# print("Download Done!!")

In [ ]:
# llm_download_model_full_path = llm_download_model_dir
# llm_model.download_model(model_filename="orca-mini-3b.ggmlv3.q4_0.bin", model_path=llm_download_model_full_path)
# print("Done!")

In [ ]:
llm_download_model_full_path = "/content/drive/MyDrive/Colab Notebooks/ds_projects/llm_model/"

In [ ]:
from gpt4all import GPT4All

In [ ]:
llm_model = GPT4All("orca-mini-3b.ggmlv3.q4_0.bin")

Found model file at  /root/.cache/gpt4all/orca-mini-3b.ggmlv3.q4_0.bin


In [ ]:
llm_model.generate("what is the capital of Germany ?")


What is the capital of Germany?
Berlin is the capital city of Germany.


'\nWhat is the capital of Germany?\nBerlin is the capital city of Germany.'

In [ ]:
#!cp /root/.cache/gpt4all/orca-mini-3b.ggmlv3.q4_0.bin /content/drive/MyDrive/Colab Notebooks/ds_projects/llm_model/

In [ ]:
output = llm_model.generate("The capital of Germany is : ", max_tokens=3)
print(output)


Berlin


LLM Ruuning Model Using Langchain - Dit not work

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
#https://github.com/hwchase17/langchain/issues/2404

In [ ]:
template = """
Question: {question}
Answer: Let's think in step by step.
"""

prompt = PromptTemplate(template=template, input_variables = ["question"])

In [ ]:
#llm_download_model_full_path

In [ ]:
callbacks = [StreamingStdOutCallbackHandler()]
llm_model = GPT4All(model="orca-mini-3b.ggmlv3.q4_0.bin", allow_download=True,
                    callbacks=callbacks, n_ctx=100, backend="gptj", verbose=True)
#llm = GPT4All(model=llm_download_model_path, n_ctx=100, backend="gptj", callbacks=callbacks, verbose=True)

Found model file at  /root/.cache/gpt4all/orca-mini-3b.ggmlv3.q4_0.bin


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm_model)

ValidationError: ignored

In [ ]:
llm_model.generate(["What is the capital of India ?"])

Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7f3f1d583640>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gpt4all/pyllmodel.py", line 323, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 



The capital city of India is New Delhi.

In [ ]:
llm_chain.run(question)

Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7f4e7ba44160>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gpt4all/pyllmodel.py", line 323, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7f4e7ba44160>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gpt4all/pyllmodel.py", line 323, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 


# Working LangChain with GPT4ALL

# Steps
## Download the model using GPT4ALL or llm to a directory
## Now use llmCPP to initialize the model using the model path
  * Download the package llama-cpp-python
## Use llm chain to run -> it works!

In [5]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 5.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.67-cp310-cp310-linux_x86_64.whl size=266388 sha256=60895232a75ce65947662579cd39b32a32c975412685f746e61d3acf9fd33cd9
  Stored in directory: /root/.cache/pip/wheels/b9/1c/63/394761fe4b898ac1090c6d2211588e88b71ab64db6e1d950af
Successfully built llama-cpp-python


In [6]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [7]:
template = """
Question: {question}
Answer: Let's think in step by step.
"""

prompt = PromptTemplate(template=template, input_variables = ["question"])

In [8]:
#!pip install llama-cpp-python
from langchain.llms import LlamaCpp

In [9]:
llm_model = GPT4All(model="orca-mini-3b.ggmlv3.q4_0.bin", allow_download=True, verbose=True)

100%|██████████| 1.93G/1.93G [00:19<00:00, 100MiB/s]


Model downloaded at:  /root/.cache/gpt4all/orca-mini-3b.ggmlv3.q4_0.bin


In [10]:
callbacks = [StreamingStdOutCallbackHandler()]

llm_cpp_model = LlamaCpp(model_path="/root/.cache/gpt4all/orca-mini-3b.ggmlv3.q4_0.bin", n_ctx=2048, callbacks=callbacks)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [11]:
llm_chain_with_cpp = LLMChain(prompt=prompt, llm=llm_cpp_model)

In [12]:
question = "What is the capital of India ?"

In [13]:
print(llm_chain_with_cpp.run(question))

Step 1: Identify the question being asked.
Question: What is the capital of India?
Step 2: Determine the answer being sought.
Answer: The capital of India is New Delhi.
Step 3: Provide the answer.
So the answer to the question "What is the capital of India?" is New Delhi.Step 1: Identify the question being asked.
Question: What is the capital of India?
Step 2: Determine the answer being sought.
Answer: The capital of India is New Delhi.
Step 3: Provide the answer.
So the answer to the question "What is the capital of India?" is New Delhi.


# Work with Agents

In [14]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [15]:
tools = load_tools(["llm-math"], llm=llm_cpp_model)

In [ ]:
agent = initialize_agent(tools, llm_cpp_model, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("What is 10 raised to 0.43")



> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


 Determine whether we can use a calculator or not
Action: If the answer can be done using a calculator, then use the calculator to get the answer. If the answer cannot be done using a calculator, then do the calculation by hand.
Action Input: The input to the calculator
Observation: If the answer can be done using a calculator, then use the calculator to get the answer. If the answer cannot be done using a calculator, then do the calculation by hand. is not a valid tool, try another one.
Thought:

Llama.generate: prefix-match hit


 I have determined that we can use a calculator to get the answer
Action: If the answer can be done using a calculator, then use the calculator to get the answer. If the answer cannot be done using a calculator, then do the calculation by hand. is not a valid tool, try another one.
Action Input: The input to the calculator
Observation: If the answer can be done using a calculator, then use the calculator to get the answer. If the answer cannot be done using a calculator, then do the calculation by hand. is not a valid tool, try another one. is not a valid tool, try another one.
Thought:

Llama.generate: prefix-match hit


KeyboardInterrupt: ignored

# Conversational Chatbot

In [16]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent

In [17]:
FORMAT_INSTRUCTIONS = """To use a tool, please use the following format:
\```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
\```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the following format(the prefix of "Thought: " and "{ai_prefix}: " are must be included):

\```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
\```
"""

In [20]:
template_llm = """
### System:
You are an AI assistant that follows instruction extremely well. Help as much as you can.

### User:
{question}

### Response:
"""

prompt = PromptTemplate(template=template_llm, input_variables = ["question"])

In [32]:
llm_chain_with_cpp = LLMChain(prompt=prompt, llm=llm_cpp_model)

In [35]:
llm_chain_with_cpp.run("What is the biggest city in the world")

Llama.generate: prefix-match hit


 The biggest city in the world by population is currently Tokyo, Japan. However, if you are looking for the biggest city by land area, it would be considered either Delhi or Shanghai.

' The biggest city in the world by population is currently Tokyo, Japan. However, if you are looking for the biggest city by land area, it would be considered either Delhi or Shanghai.'

In [104]:
input_paragraph = """
author: Matt Schlicht
claps: 5K
reading_time: 11
link: https://chatbotsmagazine.com/the-complete-beginner-s-guide-to-chatbots-8280b7b906ca?source=tag_archive---------3----------------
title: The Complete Beginner’s Guide To Chatbots – Chatbots Magazine
text: What are chatbots? Why are they such a big opportunity? How do they work? How can I build one? How can I meet other people interested in chatbots?
These are the questions we’re going to answer for you right now.
Ready? Let’s do this.
(Do you work in ecommerce? Stop reading and click here, we made something for you.)
(p.s. here is where I believe the future of bots is headed, you will probably disagree with me at first.)
(p.p.s. My newest guide about conversational commerce is up, I think you’ll find it super interesting.)
A chatbot is a service, powered by rules and sometimes artificial intelligence, that you interact with via a chat interface. The service could be any number of things, ranging from functional to fun, and it could live in any major chat product (Facebook Messenger, Slack, Telegram, Text Messages, etc.).
If you haven’t wrapped your head around it yet, don’t worry. Here’s an example to help you visualize a chatbot.
If you wanted to buy shoes from Nordstrom online, you would go to their website, look around until you find the shoes you wanted, and then you would purchase them.
If Nordstrom makes a bot, which I’m sure they will, you would simply be able to message Nordstrom on Facebook. It would ask you what you’re looking for and you would simply... tell it.
Instead of browsing a website, you will have a conversation with the Nordstrom bot, mirroring the type of experience you would get when you go into the retail store.
Watch this video from Facebook’s recent F8 conference (where they make their major announcements). At the 7:30 mark, David Marcus, the Vice President of Messaging Products at Facebook, explains what it looks like to buy shoes in a Facebook Messenger bot.
Buying shoes isn’t the only thing chatbots can be used for. Here are a couple of other examples:
See? With bots, the possibilities are endless. You can build anything imaginable, and I encourage you to do just that.
But why make a bot? Sure, it looks cool, it’s using some super advanced technology, but why should someone spend their time and energy on it?
It’s a huge opportunity. HUGE. Scroll down and I’ll explain.
You are probably wondering “Why does anyone care about chatbots? They look like simple text based services... what’s the big deal?”
Great question. I’ll tell you why people care about chatbots.
It’s because for the first time ever people are using messenger apps more than they are using social networks.
Let that sink in for a second.
People are using messenger apps more than they are using social networks.
So, logically, if you want to build a business online, you want to build where the people are. That place is now inside messenger apps.
This is why chatbots are such a big deal. It’s potentially a huge business opportunity for anyone willing to jump headfirst and build something people want.
But, how do these bots work? How do they know how to talk to people and answer questions? Isn’t that artificial intelligence and isn’t that insanely hard to do?
Yes, you are correct, it is artificial intelligence, but it’s something that you can totally do yourself.
Let me explain.
There are two types of chatbots, one functions based on a set of rules, and the other more advanced version uses machine learning.
What does this mean?


Can you please summarize this into 4 lines ?
"""

In [105]:
llm_chain_with_cpp.run(input_paragraph)

Llama.generate: prefix-match hit


KeyboardInterrupt: ignored

In [106]:
input_paragraph_small = """
You are probably wondering “Why does anyone care about chatbots? They look like simple text based services... what’s the big deal?”
Great question. I’ll tell you why people care about chatbots.
It’s because for the first time ever people are using messenger apps more than they are using social networks.
Let that sink in for a second.
People are using messenger apps more than they are using social networks.
So, logically, if you want to build a business online, you want to build where the people are. That place is now inside messenger apps.
This is why chatbots are such a big deal. It’s potentially a huge business opportunity for anyone willing to jump headfirst and build something people want.
But, how do these bots work? How do they know how to talk to people and answer questions? Isn’t that artificial intelligence and isn’t that insanely hard to do?
Yes, you are correct, it is artificial intelligence, but it’s something that you can totally do yourself.
Let me explain.
There are two types of chatbots, one functions based on a set of rules, and the other more advanced version uses machine learning.
What does this mean?


Can you please summarize this into 4 lines ?
"""

In [ ]:
llm_chain_with_cpp.run(input_paragraph_small)

Llama.generate: prefix-match hit


In [36]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [37]:
generate_tool = Tool(
    name="Language Model",
    func=llm_chain_with_cpp.run,
    description="Use this model to generate text for questions and answers"
)

In [38]:
callbacks = [StreamingStdOutCallbackHandler]

In [42]:
agent_chain = initialize_agent([],
                               llm_cpp_model,
                               agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
                               verbose=True,
                               memory=memory,
                               agent_kwargs={"format_instructions": FORMAT_INSTRUCTIONS})

In [43]:
print(agent_chain.run(input="What are types of machine learning ?"))



> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


KeyboardInterrupt: ignored

# Retriever

In [96]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

In [93]:
retriever = vector_store.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["k"] = 1

In [97]:
qa_chain = ConversationalRetrievalChain.from_llm(llm_cpp_model, retriever=retriever,  max_tokens_limit=2000)

In [ ]:
#qa_chain = RetrievalQA.from_chain_type(llm_cpp_model, retriever=retriever, chain_type="refine")

In [100]:
qa_chain.run({"question": "Summarize what are chatbots in 3 lines", "chat_history": []})

Llama.generate: prefix-match hit


KeyboardInterrupt: ignored